In [8]:
import numpy as np
import csv

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression

with open("\Documents\2951r\walking2.csv", 'rb') as f:
    reader = csv.reader(f)
    for row in reader:
        print row

IOError: [Errno 22] invalid mode ('rb') or filename: '\\Documents\x02951r\\walking2.csv'